In [1]:
##LoRA implementation with Pytorch

Yo!!


In [1]:
import torch.nn as nn
import torch

In [ ]:
## Define LoRA

class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out,rank=1,alpha=1,device="cpu"):
        super().__init__()

        self.Lora_A = nn.Parameter(torch.zeros((rank,features_out)).to(device))
        self.Lora_B = nn.Parameter(torch.zeros((features_in,rank)).to(device))
        nn.init.normal_(self.Lora_A,mean=0,std=1)

        self.scale = alpha/rank
        self.enabled = True

    def forward(self,original_weights):
        if self.enabled:
            return original_weights + torch.matmul(self.Lora_B,self.Lora_A).view(original_weights.shape)* self.scale
        else:
            return original_weights    

In [ ]:
## How to add in any network in pytorch 
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterisation(layer,device="cpu",rank=1,lora_alpha=1):
    features_in, features_out = layer.weight.shape
    return LoRAParametrization(features_in,features_out,rank=rank,alpha=lora_alpha,device=device)

parametrize.register_parametrization("your layer", "weight",linear_layer_parameterisation("your layer",device="cpu"))
parametrize.register_parametrization("your layer n ", "weight",linear_layer_parameterisation("your layer",device="cpu"))

def enable_disable_Lora(enabled=True):
    for layer in ["your layer1","your layer n"]:
        layer.parametrizations["weight"][0].enabled = enabled

In [ ]:
### For testing you can create very normal neural network which might be overfiiting for one class, now this upper LoRa will help us to finetune our network on that class